In [1]:
# takes 4h30m for 3m hand records. Create acbl_hand_records_bidding.pkl.

# todo:
# next steps:
# more experiments

# previous steps:
# acbl_hand_records_augment.ipynb creates acbl_hand_records_augmented.pkl

# Requirements
# https://github.com/Afwas/python-dds with some of my mods.

In [2]:
import pandas as pd
import pathlib
from IPython.display import display # needed to define display() method in vscode
import sys

In [3]:
sys.path.append(str(pathlib.Path.cwd().parent.joinpath('mlBridgeLib')))
sys.path
import mlBridgeLib

In [4]:
# override pandas display options
mlBridgeLib.pd_options_display()

In [5]:
rootPath = pathlib.Path('e:/bridge/data')
acblPath = rootPath.joinpath('acbl')

In [6]:
# takes 45s
acbl_hand_records_augmented_filename = 'acbl_hand_records_augmented.pkl'
acbl_hand_records_augmented_file = acblPath.joinpath(acbl_hand_records_augmented_filename)
stdhrs = pd.read_pickle(acbl_hand_records_augmented_file)
display(len(stdhrs),stdhrs)

3355150

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,DDmakes,DD_N_C,DD_N_D,DD_N_H,DD_N_S,DD_N_N,DD_E_C,DD_E_D,DD_E_H,DD_E_S,DD_E_N,DD_S_C,DD_S_D,DD_S_H,DD_S_S,DD_S_N,DD_W_C,DD_W_D,DD_W_H,DD_W_S,DD_W_N,Hands_Bin,HB_N,HB_E,HB_S,HB_W,HCP,HCP_N,HCP_N_S,HCP_N_H,HCP_N_D,HCP_N_C,HCP_E,HCP_E_S,HCP_E_H,HCP_E_D,HCP_E_C,HCP_S,HCP_S_S,HCP_S_H,HCP_S_D,HCP_S_C,HCP_W,HCP_W_S,HCP_W_H,HCP_W_D,HCP_W_C,QT,QT_N,QT_N_S,...,CT_NS_H,CT_NS_S,CT_NS_N,CT_EW_C,CT_EW_D,CT_EW_H,CT_EW_S,CT_EW_N,CT_NS_C_Pass,CT_NS_C_Partial,CT_NS_C_Game,CT_NS_C_SSlam,CT_NS_C_GSlam,CT_NS_D_Pass,CT_NS_D_Partial,CT_NS_D_Game,CT_NS_D_SSlam,CT_NS_D_GSlam,CT_NS_H_Pass,CT_NS_H_Partial,CT_NS_H_Game,CT_NS_H_SSlam,CT_NS_H_GSlam,CT_NS_S_Pass,CT_NS_S_Partial,CT_NS_S_Game,CT_NS_S_SSlam,CT_NS_S_GSlam,CT_NS_N_Pass,CT_NS_N_Partial,CT_NS_N_Game,CT_NS_N_SSlam,CT_NS_N_GSlam,CT_EW_C_Pass,CT_EW_C_Partial,CT_EW_C_Game,CT_EW_C_SSlam,CT_EW_C_GSlam,CT_EW_D_Pass,CT_EW_D_Partial,CT_EW_D_Game,CT_EW_D_SSlam,CT_EW_D_GSlam,CT_EW_H_Pass,CT_EW_H_Partial,CT_EW_H_Game,CT_EW_H_SSlam,CT_EW_H_GSlam,CT_EW_S_Pass,CT_EW_S_Partial,CT_EW_S_Game,CT_EW_S_SSlam,CT_EW_S_GSlam,CT_EW_N_Pass,CT_EW_N_Partial,CT_EW_N_Game,CT_EW_N_SSlam,CT_EW_N_GSlam,Vul_NS,Vul_EW
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))",10,4,10,4,8,2,9,3,9,2,10,4,10,4,8,2,9,3,9,2,"((0b100010000010000000001100010000000011001011100100, (0b100010000, 0b100000000011, 0b1000000001...",0b100010000010000000001100010000000011001011100100,0b1000001001010001000000000001100100100100100000001001,0b100000100000100111100010010000000010000000100010010,0b11010000101000000011100000001011001000010000000000,"((9, (0, 3, 1, 5)), (14, (4, 2, 5, 3)), (12, (3, 5, 4, 0)), (5, (3, 0, 0, 2)))",9,0,3,1,5,14,4,2,5,3,12,3,5,4,0,5,3,0,0,2,"(6.5, ((1.5, (0, 0.5, 0, 1)), (2.5, (1, 0, 1, 0.5)), (2.5, (0.5, 1, 1, 0)), (0, (0, 0, 0, 0))))",1.50,0.00,...,Game,Pass,Partial,Pass,Partial,Pass,Partial,Pass,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))",8,8,7,7,7,4,5,6,6,6,8,8,7,7,7,4,5,6,6,6,"((0b101001000010100000000000010000001100101001100010, (0b101001000, 0b101000000000, 0b1000000110...",0b101001000010100000000000010000001100101001100010,0b1001000010100000011100000000001001010001000000001001,0b100010100000001000010100110100010000010010000010000,0b10000000011100000001011001000100100000000110000100,"((9, (0, 4, 1, 4)), (9, (5, 0, 0, 4)), (13, (3, 2, 6, 2)), (9, (2, 4, 3, 0)))",9,0,4,1,4,9,5,0,0,4,13,3,2,6,2,9,2,4,3,0,"(6.5, ((1.0, (0, 0.5, 0, 0.5)), (2, (1, 0, 0, 1)), (2.0, (0.5, 0, 1.5, 0)), (1.5, (0, 1, 0.5, 0))))",1.00,0.00,...,Partial,Partial,Partial,Pass,Pass,Pass,Pass,Pass,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJTD65CAKTSK984H5DT8743C987SA52HAQ9832DKJC42,"((T, 764, AQ92, QJ653), (K984, 5, T8743, 987), (A52, AQ9832, KJ, 42), (QJ763, KJT, 65, AKT))","(140, [(3

In [7]:
# takes 45s + 1m + 1m
# since computing DDmakes and Par is computationally slow, attempt to reuse previous computations.
# todo: code similar to acbl_hand_records_cleanup
acbl_hand_records_dd_nwse_experiments_filename = 'acbl_hand_records_dd_nwse_experiments.pkl'
acbl_hand_records_dd_nwse_experiments_file = acblPath.joinpath(acbl_hand_records_dd_nwse_experiments_filename)
if acbl_hand_records_dd_nwse_experiments_file.exists():
    hrcdf = pd.read_pickle(acbl_hand_records_dd_nwse_experiments_file)
    display(len(hrcdf),hrcdf.head())
    # Preferred to use pd.merge but caused "out of memory" condition. Experiment with using dicts instead.
    DDmakesd = dict(zip(hrcdf['board_record_string'],hrcdf['DDmakes_nwse']))
    Pard = dict(zip(hrcdf['board_record_string'],hrcdf['Par_nwse']))
    #display(DDmakesd)
    #stdhrs = pd.merge(stdhrs,hrcdf[['EventBoard','DDmakes','Par']],on='EventBoard',how='left')
    #display(hrcdf[hrcdf['EventBoard'].isin(stdhrs['EventBoard'])]
    #display(len(stdhrs),stdhrs)
    #stdhrs.drop(['DDmakes_nwse','Par_nwse'],axis='columns',inplace=True)
    if 'DDmakes_nwse' not in stdhrs:
        stdhrs.insert(stdhrs.columns.get_loc('DDmakes')+1,'DDmakes_nwse',None)
        #stdhrs['DDmakes_nwse'] = pd.DataFrame(tuple((0,[0]))) # stdhrs['DDmakes_nwse'].astype('object')
    if 'Par_nwse' not in stdhrs:
        stdhrs.insert(stdhrs.columns.get_loc('Par')+1,'Par_nwse',None)
        #stdhrs['Par_nwse'] = pd.DataFrame(tuple((0,[0]))) # stdhrs['DDmakes_nwse'].astype('object')    hrcdf = pd.read_pickle(acbl_hand_records_dd_experiments_file)
    # takes 1m
    DDmakesClean = stdhrs.apply(lambda r: DDmakesd[r['board_record_string']] if r['board_record_string'] in DDmakesd else None,axis='columns')
    stdhrs.loc[stdhrs['DDmakes_nwse'].isna(),'DDmakes_nwse'] = DDmakesClean
    b =  DDmakesClean.isna() | stdhrs['DDmakes_nwse'].eq(DDmakesClean)
    assert all(b)
    # takes 1m
    # verify that dirty Pars are still dirty
    ParClean = stdhrs.apply(lambda r: Pard[r['board_record_string']] if r['board_record_string'] in Pard else None,axis='columns')
    b = ParClean.notna() & stdhrs['Par_nwse'].ne(ParClean)
    display(stdhrs[b])
    # assign clean Pars. verify that all Pars are now cleaned.
    stdhrs.loc[b,'Par_nwse'] = ParClean
    b = ParClean.isna() | stdhrs['Par_nwse'].eq(ParClean)
    assert all(b)
else:
    ParClean = pd.Series()

3144755

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,Par_nwse,DDmakes,DDmakes_nwse,DD_N_C,DD_N_D,DD_N_H,DD_N_S,DD_N_N,DD_E_C,DD_E_D,DD_E_H,DD_E_S,DD_E_N,DD_S_C,DD_S_D,DD_S_H,DD_S_S,DD_S_N,DD_W_C,DD_W_D,DD_W_H,DD_W_S,DD_W_N,Hands_Bin,HB_N,HB_E,HB_S,HB_W,HCP,HCP_N,HCP_N_S,HCP_N_H,HCP_N_D,HCP_N_C,HCP_E,HCP_E_S,HCP_E_H,HCP_E_D,HCP_E_C,HCP_S,HCP_S_S,HCP_S_H,HCP_S_D,HCP_S_C,HCP_W,HCP_W_S,HCP_W_H,HCP_W_D,HCP_W_C,QT,...,CT_NS_H,CT_NS_S,CT_NS_N,CT_EW_C,CT_EW_D,CT_EW_H,CT_EW_S,CT_EW_N,CT_NS_C_Pass,CT_NS_C_Partial,CT_NS_C_Game,CT_NS_C_SSlam,CT_NS_C_GSlam,CT_NS_D_Pass,CT_NS_D_Partial,CT_NS_D_Game,CT_NS_D_SSlam,CT_NS_D_GSlam,CT_NS_H_Pass,CT_NS_H_Partial,CT_NS_H_Game,CT_NS_H_SSlam,CT_NS_H_GSlam,CT_NS_S_Pass,CT_NS_S_Partial,CT_NS_S_Game,CT_NS_S_SSlam,CT_NS_S_GSlam,CT_NS_N_Pass,CT_NS_N_Partial,CT_NS_N_Game,CT_NS_N_SSlam,CT_NS_N_GSlam,CT_EW_C_Pass,CT_EW_C_Partial,CT_EW_C_Game,CT_EW_C_SSlam,CT_EW_C_GSlam,CT_EW_D_Pass,CT_EW_D_Partial,CT_EW_D_Game,CT_EW_D_SSlam,CT_EW_D_GSlam,CT_EW_H_Pass,CT_EW_H_Partial,CT_EW_H_Game,CT_EW_H_SSlam,CT_EW_H_GSlam,CT_EW_S_Pass,CT_EW_S_Partial,CT_EW_S_Game,CT_EW_S_SSlam,CT_EW_S_GSlam,CT_EW_N_Pass,CT_EW_N_Partial,CT_EW_N_Game,CT_EW_N_SSlam,CT_EW_N_GSlam,Vul_NS,Vul_EW
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])","(-100, [(5, C, *, S, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))","((9, 3, 9, 3, 8), (2, 10, 3, 10, 2), (10, 3, 9, 3, 8), (2, 10, 3, 10, 2))",10,4,10,4,8,2,9,3,9,2,10,4,10,4,8,2,9,3,9,2,"((0b100010000010000000001100010000000011001011100100, (0b100010000, 0b100000000011, 0b1000000001...",0b100010000010000000001100010000000011001011100100,0b1000001001010001000000000001100100100100100000001001,0b100000100000100111100010010000000010000000100010010,0b11010000101000000011100000001011001000010000000000,"((9, (0, 3, 1, 5)), (14, (4, 2, 5, 3)), (12, (3, 5, 4, 0)), (5, (3, 0, 0, 2)))",9,0,3,1,5,14,4,2,5,3,12,3,5,4,0,5,3,0,0,2,"(6.5, ((1.5, (0, 0.5, 0, 1)), (2.5, (1, 0, 1, 0.5)), (2.5, (0.5, 1, 1, 0)), (0, (0, 0, 0, 0))))",...,Game,Pass,Partial,Pass,Partial,Pass,Partial,Pass,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","(90, [(1, N, , NS, 0), (1, D, , NS, 1), (1, C, , NS, 1)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))","((8, 8, 7, 7, 7), (4, 5, 6, 6, 5), (8, 8, 7, 7, 7), (4, 5, 6, 6, 5))",8,8,7,7,7,4,5,6,6,6,8,8,7,7,7,4,5,6,6,6,"((0b101001000010100000000000010000001100101001100010, (0b101001000, 0b101000000000, 0b1000000110...",0b101001000010100000000000010000001100101001100010,0b1001000010100000011100000000001001010001000000001001,0b100010100000001000010100110100010000010010000010000,0b10000000011100000001011001000100100000000110000100,"((9, (0, 4, 1, 4)), (9, (5, 0, 0, 4)), (13, (3, 2, 6, 2)), (9, (2, 4, 3, 0)))",9,0,4,1,4,9,5,0,0,4,13,3,2,6,2,9,2,4,3,0,"(6.5, ((1.0, (0, 0.5, 0, 0.5)), (2, (1, 0, 0, 1)), (2.0, (0.5, 0, 1.5, 0)), (1.5, (0, 1, 0.5, 0))))",...,Partial,Partial,Partial,Pass,Pass,Pass,Pass,Pass,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True
2

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,Par_nwse,DDmakes,DDmakes_nwse,DD_N_C,DD_N_D,DD_N_H,DD_N_S,DD_N_N,DD_E_C,DD_E_D,DD_E_H,DD_E_S,DD_E_N,DD_S_C,DD_S_D,DD_S_H,DD_S_S,DD_S_N,DD_W_C,DD_W_D,DD_W_H,DD_W_S,DD_W_N,Hands_Bin,HB_N,HB_E,HB_S,HB_W,HCP,HCP_N,HCP_N_S,HCP_N_H,HCP_N_D,HCP_N_C,HCP_E,HCP_E_S,HCP_E_H,HCP_E_D,HCP_E_C,HCP_S,HCP_S_S,HCP_S_H,HCP_S_D,HCP_S_C,HCP_W,HCP_W_S,HCP_W_H,HCP_W_D,HCP_W_C,QT,...,CT_NS_H,CT_NS_S,CT_NS_N,CT_EW_C,CT_EW_D,CT_EW_H,CT_EW_S,CT_EW_N,CT_NS_C_Pass,CT_NS_C_Partial,CT_NS_C_Game,CT_NS_C_SSlam,CT_NS_C_GSlam,CT_NS_D_Pass,CT_NS_D_Partial,CT_NS_D_Game,CT_NS_D_SSlam,CT_NS_D_GSlam,CT_NS_H_Pass,CT_NS_H_Partial,CT_NS_H_Game,CT_NS_H_SSlam,CT_NS_H_GSlam,CT_NS_S_Pass,CT_NS_S_Partial,CT_NS_S_Game,CT_NS_S_SSlam,CT_NS_S_GSlam,CT_NS_N_Pass,CT_NS_N_Partial,CT_NS_N_Game,CT_NS_N_SSlam,CT_NS_N_GSlam,CT_EW_C_Pass,CT_EW_C_Partial,CT_EW_C_Game,CT_EW_C_SSlam,CT_EW_C_GSlam,CT_EW_D_Pass,CT_EW_D_Partial,CT_EW_D_Game,CT_EW_D_SSlam,CT_EW_D_GSlam,CT_EW_H_Pass,CT_EW_H_Partial,CT_EW_H_Game,CT_EW_H_SSlam,CT_EW_H_GSlam,CT_EW_S_Pass,CT_EW_S_Partial,CT_EW_S_Game,CT_EW_S_SSlam,CT_EW_S_GSlam,CT_EW_N_Pass,CT_EW_N_Partial,CT_EW_N_Game,CT_EW_N_SSlam,CT_EW_N_GSlam,Vul_NS,Vul_EW
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])",None,"((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))","((9, 3, 9, 3, 8), (2, 10, 3, 10, 2), (10, 3, 9, 3, 8), (2, 10, 3, 10, 2))",10,4,10,4,8,2,9,3,9,2,10,4,10,4,8,2,9,3,9,2,"((0b100010000010000000001100010000000011001011100100, (0b100010000, 0b100000000011, 0b1000000001...",0b100010000010000000001100010000000011001011100100,0b1000001001010001000000000001100100100100100000001001,0b100000100000100111100010010000000010000000100010010,0b11010000101000000011100000001011001000010000000000,"((9, (0, 3, 1, 5)), (14, (4, 2, 5, 3)), (12, (3, 5, 4, 0)), (5, (3, 0, 0, 2)))",9,0,3,1,5,14,4,2,5,3,12,3,5,4,0,5,3,0,0,2,"(6.5, ((1.5, (0, 0.5, 0, 1)), (2.5, (1, 0, 1, 0.5)), (2.5, (0.5, 1, 1, 0)), (0, (0, 0, 0, 0))))",...,Game,Pass,Partial,Pass,Partial,Pass,Partial,Pass,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])",None,"((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))","((8, 8, 7, 7, 7), (4, 5, 6, 6, 5), (8, 8, 7, 7, 7), (4, 5, 6, 6, 5))",8,8,7,7,7,4,5,6,6,6,8,8,7,7,7,4,5,6,6,6,"((0b101001000010100000000000010000001100101001100010, (0b101001000, 0b101000000000, 0b1000000110...",0b101001000010100000000000010000001100101001100010,0b1001000010100000011100000000001001010001000000001001,0b100010100000001000010100110100010000010010000010000,0b10000000011100000001011001000100100000000110000100,"((9, (0, 4, 1, 4)), (9, (5, 0, 0, 4)), (13, (3, 2, 6, 2)), (9, (2, 4, 3, 0)))",9,0,4,1,4,9,5,0,0,4,13,3,2,6,2,9,2,4,3,0,"(6.5, ((1.0, (0, 0.5, 0, 0.5)), (2, (1, 0, 0, 1)), (2.0, (0.5, 0, 1.5, 0)), (1.5, (0, 1, 0.5, 0))))",...,Partial,Partial,Partial,Pass,Pass,Pass,Pass,Pass,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True
2,"(acbl, 2620411)",77908,100961,19,200217A_19,S,E_W,STH764DAQ92CQJ653SQJ763HKJT

In [8]:
# undertricks: 0 = make; 1-13 = sacrifice
# overTricks: 0-3; e.g. 1 for 4S + 1
# level: 1-7
denom = 'NSHDC' # DDS ordering
seats = ['N','E','S','W','NS','EW'] # Par score player's direction codes

# todo: change references to use mlBridgeLib
dealer_d = {'N':0, 'E':1, 'S':2, 'W':3}
vul_d = {'None':0, 'Both':1, 'N_S':2, 'E_W':3} # dds vul encoding is weird

In [12]:
# perform initialization for calculation of double dummy (all positions and all strains) and Pars.

import dds
import functions
import ctypes

starting_handno = 0 # for restarting at a particular hando (0-len(stdhrs)). normally 0.

DDdealsPBN = dds.ddTableDealsPBN()
tableRes = dds.ddTablesRes()
pres = dds.allParResults()
presmaster = dds.parResultsMaster()

mode = 0
tFilter = ctypes.c_int * dds.DDS_STRAINS
trumpFilter = tFilter(0, 0, 0, 0, 0)
line = ctypes.create_string_buffer(80)

dds.SetMaxThreads(0)
max_tables = dds.MAXNOOFTABLES

df = stdhrs[ParClean.isna()]
nhands = len(df)
checkpoint_after_every = 1000
df

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,Par_nwse,DDmakes,DDmakes_nwse,DD_N_C,DD_N_D,DD_N_H,DD_N_S,DD_N_N,DD_E_C,DD_E_D,DD_E_H,DD_E_S,DD_E_N,DD_S_C,DD_S_D,DD_S_H,DD_S_S,DD_S_N,DD_W_C,DD_W_D,DD_W_H,DD_W_S,DD_W_N,Hands_Bin,HB_N,HB_E,HB_S,HB_W,HCP,HCP_N,HCP_N_S,HCP_N_H,HCP_N_D,HCP_N_C,HCP_E,HCP_E_S,HCP_E_H,HCP_E_D,HCP_E_C,HCP_S,HCP_S_S,HCP_S_H,HCP_S_D,HCP_S_C,HCP_W,HCP_W_S,HCP_W_H,HCP_W_D,HCP_W_C,QT,...,CT_NS_H,CT_NS_S,CT_NS_N,CT_EW_C,CT_EW_D,CT_EW_H,CT_EW_S,CT_EW_N,CT_NS_C_Pass,CT_NS_C_Partial,CT_NS_C_Game,CT_NS_C_SSlam,CT_NS_C_GSlam,CT_NS_D_Pass,CT_NS_D_Partial,CT_NS_D_Game,CT_NS_D_SSlam,CT_NS_D_GSlam,CT_NS_H_Pass,CT_NS_H_Partial,CT_NS_H_Game,CT_NS_H_SSlam,CT_NS_H_GSlam,CT_NS_S_Pass,CT_NS_S_Partial,CT_NS_S_Game,CT_NS_S_SSlam,CT_NS_S_GSlam,CT_NS_N_Pass,CT_NS_N_Partial,CT_NS_N_Game,CT_NS_N_SSlam,CT_NS_N_GSlam,CT_EW_C_Pass,CT_EW_C_Partial,CT_EW_C_Game,CT_EW_C_SSlam,CT_EW_C_GSlam,CT_EW_D_Pass,CT_EW_D_Partial,CT_EW_D_Game,CT_EW_D_SSlam,CT_EW_D_GSlam,CT_EW_H_Pass,CT_EW_H_Partial,CT_EW_H_Game,CT_EW_H_SSlam,CT_EW_H_GSlam,CT_EW_S_Pass,CT_EW_S_Partial,CT_EW_S_Game,CT_EW_S_SSlam,CT_EW_S_GSlam,CT_EW_N_Pass,CT_EW_N_Partial,CT_EW_N_Game,CT_EW_N_SSlam,CT_EW_N_GSlam,Vul_NS,Vul_EW
25756,"(acbl, 6996597)",275972,381367,04,210901A_04,W,Both,SAJ864HQ3DQ6CAQ95ST93HAK9DT83CJT72SQ5HT762DJ97C8643SK72HJ854DAK542CK,"((AJ864, Q3, Q6, AQ95), (Q5, T762, J97, 8643), (K72, J854, AK542, K), (T93, AK9, T83, JT72))","(660, [(3, N, , NS, 2)])","(660, [(3, N, , NS, 2)])","((10, 11, 10, 11, 11), (3, 0, 2, 0, 0), (10, 11, 10, 11, 11), (3, 0, 2, 0, 0))","((10, 11, 10, 11, 11), (3, 0, 3, 0, 0), (10, 11, 10, 11, 11), (3, 0, 3, 0, 0))",10,11,10,11,11,3,0,2,0,0,10,11,10,11,11,3,0,2,0,0,"((0b1001001010100001000000001000100000100001010010001000, (0b1001001010100, 0b10000000010, 0b100...",0b1001001010100001000000001000100000100001010010001000,0b10000001000000010011000100010101000000000001010110,0b100000100001000100100110011000000011010100000000000,0b110000010110001000000000001010000100001100100001,"((15, (5, 2, 2, 6)), (3, (2, 0, 1, 0)), (14, (3, 1, 7, 3)), (8, (0, 7, 0, 1)))",15,5,2,2,6,3,2,0,1,0,14,3,1,7,3,8,0,7,0,1,"(7.5, ((2.5, (1, 0, 0, 1.5)), (0, (0, 0, 0, 0)), (3.0, (0.5, 0, 2, 0.5)), (2, (0, 2, 0, 0))))",...,Game,Game,Game,Pass,Pass,Pass,Pass,Pass,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True
25757,"(acbl, 6996600)",275972,381367,07,210901A_07,S,Both,SAHAKQ6DA6CAQJ965SQ732H873DQ9753C8ST96HJ94DKJTCKT43SKJ854HT52D842C72,"((A, AKQ6, A6, AQJ965), (T96, J94, KJT, KT43), (KJ854, T52, 842, 72), (Q732, 873, Q9753, 8))","(620, [(4, H, , NS, 0)])","(650, [(4, H, , NS, 1)])","((10, 7, 10, 9, 8), (3, 6, 2, 4, 5), (10, 7, 10, 9, 8), (3, 6, 2, 4, 5))","((10, 8, 11, 9, 8), (2, 5, 1, 3, 2), (10, 8, 11, 9, 8), (3, 5, 1, 3, 5))",10,7,10,9,8,3,6,2,4,5,10,7,10,9,8,3,6,2,4,5,"((0b1000000000000111000001000010000000100001011010011000, (0b1000000000000, 0b1110000010000, 0b1...",0b1000000000000111000001000010000000100001011010011000,0b110010000000101000010001011000000000100100000110,0b101001001100000010000100100000010001010000000100001,0b10000100011000000110001000100101010100000001000000,"((24, (4, 9, 4, 7)), (8, (0, 1, 4, 3)), (4, (4, 0, 0, 0)), (4, (2, 0, 2, 0)))",24,4,9,4,7,8,0,1,4,3,4,4,0,0,0,4,2,0,2,0,"(7.0, ((5.5, (1, 2, 1, 1.5)), (1.0, (0, 0, 0.5, 0.5)), (0.5, (0.5, 0, 0, 0)), (0, (0, 0, 0, 0))))",...,Game,Partial,Partial,Pass,Pass,Pass,Pass,Pass,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True
25758,"(acbl, 6996603)",275972,381367,10,210901A

In [13]:
# takes 1h per 100k hands.
# Perform dd and par calcs. Restartable.
# todo: implement direction in pbn. always using 'N:' for now.
# todo: some pars in acbl_hand_records.pkl are incorrect due to ignoring of vul. Need to correct pkl in previous step.
import numpy as np

for n,grp_start in enumerate(range(starting_handno,nhands,max_tables)):
    n += 1
    grp_count = min(nhands-grp_start,max_tables)
    DDdealsPBN.noOfTables = grp_count
    print(f'Processing group:{n} hands:{grp_start} to {grp_start+grp_count} of {nhands}')
    for handno in range(grp_count):
        r = df.iloc[grp_start+handno]
        nwse_tuple = tuple(r['Hands'][hand] for hand in [0,3,2,1])
        pbn = 'N:'+' '.join('.'.join([suit for suit in suits]) for suits in nwse_tuple)
        assert len(pbn) == 1+1+13*4+3*4+3, len(pbn) # 69=='N'+':'+(13 cards per hand)+(3 '.' per suit per hand)+3 spaces
        pbn_encoded = pbn.encode()
        #print(len(pbn),pbn,pbn_encoded)
        DDdealsPBN.deals[handno].cards = pbn_encoded

    # CalcAllTablesPBN will do multi-threading
    res = dds.CalcAllTablesPBN(ctypes.pointer(DDdealsPBN), mode, trumpFilter, ctypes.pointer(tableRes), ctypes.pointer(pres))

    if res != dds.RETURN_NO_FAULT:
        dds.ErrorMessage(res, line)
        print(f'CalcAllTablesPBN: DDS error: {line.value.decode("utf-8")}')
        assert False, grp_start
    
    for handno in range(0, grp_count):
        r = df.iloc[grp_start+handno]
        #display(r)
        ref = r['ref'][1]
        dealer = r['Dealer']
        assert dealer in 'NESW'
        vul = r['Vul']
        assert vul in ['None','N_S','E_W','Both'], r
        par = r['Par']
        assert type(par) is tuple, r
        assert len(par) == 2, r
        assert type(par[0]) is int, r
        assert type(par[1]) is list, r
        assert len(par[1]) > 0, r
        par_result = ctypes.pointer(presmaster)
        dd_result = ctypes.pointer(tableRes.results[handno])

        # Par calculations are not multi-threading
        res = dds.DealerParBin(dd_result, par_result, dealer_d[dealer], vul_d[vul])
        if res != dds.RETURN_NO_FAULT:
            dds.ErrorMessage(res, line)
            print(f'DealerParBin: DDS error: {line.value.decode("utf-8")}')
            assert False, r

        score = par_result.contents.score
        if score == 0:
            pars_nwse = (0, [(0, '', '', '', 0)]) # par score is for everyone to pass (1 out of 100,000)
        else:
            assert par_result.contents.number > 0, r
            pars_nwse = (score,[])
            for i in range(par_result.contents.number):
                ct = par_result.contents.contracts[i]    
                #print(f'Par[{i}]: underTricks:{ct.underTricks} overTricks:{ct.overTricks} level:{ct.level} denom:{ct.denom} seats:{ct.seats}')
                assert ct.underTricks == 0 or ct.overTricks == 0
                pars_nwse[1].append((ct.level,denom[ct.denom],'*' if ct.underTricks else '',seats[ct.seats],ct.overTricks-ct.underTricks))

        #functions.PrintPBNHand(f'Group:{n} Hand:{grp_start+handno+1}/{nhands} ACBL Ref:{ref} Vul:{vul}', DDdealsPBN.deals[handno].cards)
        #functions.PrintTable(dd_result)
        DDtable_nwse = tuple(tuple(dd_result.contents.resTable[suit][pl] for suit in [3,2,1,0,4]) for pl in range(4))
        stdhrs.at[r.name,'DDmakes_nwse'] = DDtable_nwse # updating must use stdhrs.at instead of r.iloc or r.at.
        #print(type(np.array(pars_nwse)),np.array(pars_nwse))
        stdhrs.at[r.name,'Par_nwse'] = pars_nwse # updating must use stdhrs.at instead of r.iloc or r.at.
        #display(df.head()) #.iloc[grp_start+handno])
    if n%checkpoint_after_every == 0: # save every 1000 groups
        display(df.iloc[grp_start+handno])
        acbl_hand_records_dd_nwse_experiments_filename = 'acbl_hand_records_dd_nwse_experiments.pkl'
        acbl_hand_records_dd_nwse_experiments_file = acblPath.joinpath(acbl_hand_records_dd_nwse_experiments_filename)
        df.to_pickle(acbl_hand_records_dd_nwse_experiments_file) # takes 40s
        print(f'Checkpoint at group {n}: df written to {acbl_hand_records_dd_nwse_experiments_file}')
        starting_handno = grp_start
starting_handno = nhands

Processing group:1 hands:0 to 40 of 185336
Processing group:2 hands:40 to 80 of 185336
Processing group:3 hands:80 to 120 of 185336
Processing group:4 hands:120 to 160 of 185336
Processing group:5 hands:160 to 200 of 185336
Processing group:6 hands:200 to 240 of 185336
Processing group:7 hands:240 to 280 of 185336
Processing group:8 hands:280 to 320 of 185336
Processing group:9 hands:320 to 360 of 185336
Processing group:10 hands:360 to 400 of 185336
Processing group:11 hands:400 to 440 of 185336
Processing group:12 hands:440 to 480 of 185336
Processing group:13 hands:480 to 520 of 185336
Processing group:14 hands:520 to 560 of 185336
Processing group:15 hands:560 to 600 of 185336
Processing group:16 hands:600 to 640 of 185336
Processing group:17 hands:640 to 680 of 185336
Processing group:18 hands:680 to 720 of 185336
Processing group:19 hands:720 to 760 of 185336
Processing group:20 hands:760 to 800 of 185336
Processing group:21 hands:800 to 840 of 185336
Processing group:22 hands:84

Processing group:169 hands:6720 to 6760 of 185336
Processing group:170 hands:6760 to 6800 of 185336
Processing group:171 hands:6800 to 6840 of 185336
Processing group:172 hands:6840 to 6880 of 185336
Processing group:173 hands:6880 to 6920 of 185336
Processing group:174 hands:6920 to 6960 of 185336
Processing group:175 hands:6960 to 7000 of 185336
Processing group:176 hands:7000 to 7040 of 185336
Processing group:177 hands:7040 to 7080 of 185336
Processing group:178 hands:7080 to 7120 of 185336
Processing group:179 hands:7120 to 7160 of 185336
Processing group:180 hands:7160 to 7200 of 185336
Processing group:181 hands:7200 to 7240 of 185336
Processing group:182 hands:7240 to 7280 of 185336
Processing group:183 hands:7280 to 7320 of 185336
Processing group:184 hands:7320 to 7360 of 185336
Processing group:185 hands:7360 to 7400 of 185336
Processing group:186 hands:7400 to 7440 of 185336
Processing group:187 hands:7440 to 7480 of 185336
Processing group:188 hands:7480 to 7520 of 185336


Processing group:330 hands:13160 to 13200 of 185336
Processing group:331 hands:13200 to 13240 of 185336
Processing group:332 hands:13240 to 13280 of 185336
Processing group:333 hands:13280 to 13320 of 185336
Processing group:334 hands:13320 to 13360 of 185336
Processing group:335 hands:13360 to 13400 of 185336
Processing group:336 hands:13400 to 13440 of 185336
Processing group:337 hands:13440 to 13480 of 185336
Processing group:338 hands:13480 to 13520 of 185336
Processing group:339 hands:13520 to 13560 of 185336
Processing group:340 hands:13560 to 13600 of 185336
Processing group:341 hands:13600 to 13640 of 185336
Processing group:342 hands:13640 to 13680 of 185336
Processing group:343 hands:13680 to 13720 of 185336
Processing group:344 hands:13720 to 13760 of 185336
Processing group:345 hands:13760 to 13800 of 185336
Processing group:346 hands:13800 to 13840 of 185336
Processing group:347 hands:13840 to 13880 of 185336
Processing group:348 hands:13880 to 13920 of 185336
Processing g

Processing group:488 hands:19480 to 19520 of 185336
Processing group:489 hands:19520 to 19560 of 185336
Processing group:490 hands:19560 to 19600 of 185336
Processing group:491 hands:19600 to 19640 of 185336
Processing group:492 hands:19640 to 19680 of 185336
Processing group:493 hands:19680 to 19720 of 185336
Processing group:494 hands:19720 to 19760 of 185336
Processing group:495 hands:19760 to 19800 of 185336
Processing group:496 hands:19800 to 19840 of 185336
Processing group:497 hands:19840 to 19880 of 185336
Processing group:498 hands:19880 to 19920 of 185336
Processing group:499 hands:19920 to 19960 of 185336
Processing group:500 hands:19960 to 20000 of 185336
Processing group:501 hands:20000 to 20040 of 185336
Processing group:502 hands:20040 to 20080 of 185336
Processing group:503 hands:20080 to 20120 of 185336
Processing group:504 hands:20120 to 20160 of 185336
Processing group:505 hands:20160 to 20200 of 185336
Processing group:506 hands:20200 to 20240 of 185336
Processing g

Processing group:646 hands:25800 to 25840 of 185336
Processing group:647 hands:25840 to 25880 of 185336
Processing group:648 hands:25880 to 25920 of 185336
Processing group:649 hands:25920 to 25960 of 185336
Processing group:650 hands:25960 to 26000 of 185336
Processing group:651 hands:26000 to 26040 of 185336
Processing group:652 hands:26040 to 26080 of 185336
Processing group:653 hands:26080 to 26120 of 185336
Processing group:654 hands:26120 to 26160 of 185336
Processing group:655 hands:26160 to 26200 of 185336
Processing group:656 hands:26200 to 26240 of 185336
Processing group:657 hands:26240 to 26280 of 185336
Processing group:658 hands:26280 to 26320 of 185336
Processing group:659 hands:26320 to 26360 of 185336
Processing group:660 hands:26360 to 26400 of 185336
Processing group:661 hands:26400 to 26440 of 185336
Processing group:662 hands:26440 to 26480 of 185336
Processing group:663 hands:26480 to 26520 of 185336
Processing group:664 hands:26520 to 26560 of 185336
Processing g

...

Checkpoint at group 4000: df written to e:\bridge\data\acbl\acbl_hand_records_dd_nwse_experiments.pkl
Processing group:4001 hands:160000 to 160040 of 185336
Processing group:4002 hands:160040 to 160080 of 185336
Processing group:4003 hands:160080 to 160120 of 185336
Processing group:4004 hands:160120 to 160160 of 185336
Processing group:4005 hands:160160 to 160200 of 185336
Processing group:4006 hands:160200 to 160240 of 185336
Processing group:4007 hands:160240 to 160280 of 185336
Processing group:4008 hands:160280 to 160320 of 185336
Processing group:4009 hands:160320 to 160360 of 185336
Processing group:4010 hands:160360 to 160400 of 185336
Processing group:4011 hands:160400 to 160440 of 185336
Processing group:4012 hands:160440 to 160480 of 185336
Processing group:4013 hands:160480 to 160520 of 185336
Processing group:4014 hands:160520 to 160560 of 185336
Processing group:4015 hands:160560 to 160600 of 185336
Processing group:4016 hands:160600 to 160640 of 185336
Processing group:4

Processing group:4149 hands:165920 to 165960 of 185336
Processing group:4150 hands:165960 to 166000 of 185336
Processing group:4151 hands:166000 to 166040 of 185336
Processing group:4152 hands:166040 to 166080 of 185336
Processing group:4153 hands:166080 to 166120 of 185336
Processing group:4154 hands:166120 to 166160 of 185336
Processing group:4155 hands:166160 to 166200 of 185336
Processing group:4156 hands:166200 to 166240 of 185336
Processing group:4157 hands:166240 to 166280 of 185336
Processing group:4158 hands:166280 to 166320 of 185336
Processing group:4159 hands:166320 to 166360 of 185336
Processing group:4160 hands:166360 to 166400 of 185336
Processing group:4161 hands:166400 to 166440 of 185336
Processing group:4162 hands:166440 to 166480 of 185336
Processing group:4163 hands:166480 to 166520 of 185336
Processing group:4164 hands:166520 to 166560 of 185336
Processing group:4165 hands:166560 to 166600 of 185336
Processing group:4166 hands:166600 to 166640 of 185336
Processing

Processing group:4298 hands:171880 to 171920 of 185336
Processing group:4299 hands:171920 to 171960 of 185336
Processing group:4300 hands:171960 to 172000 of 185336
Processing group:4301 hands:172000 to 172040 of 185336
Processing group:4302 hands:172040 to 172080 of 185336
Processing group:4303 hands:172080 to 172120 of 185336
Processing group:4304 hands:172120 to 172160 of 185336
Processing group:4305 hands:172160 to 172200 of 185336
Processing group:4306 hands:172200 to 172240 of 185336
Processing group:4307 hands:172240 to 172280 of 185336
Processing group:4308 hands:172280 to 172320 of 185336
Processing group:4309 hands:172320 to 172360 of 185336
Processing group:4310 hands:172360 to 172400 of 185336
Processing group:4311 hands:172400 to 172440 of 185336
Processing group:4312 hands:172440 to 172480 of 185336
Processing group:4313 hands:172480 to 172520 of 185336
Processing group:4314 hands:172520 to 172560 of 185336
Processing group:4315 hands:172560 to 172600 of 185336
Processing

Processing group:4447 hands:177840 to 177880 of 185336
Processing group:4448 hands:177880 to 177920 of 185336
Processing group:4449 hands:177920 to 177960 of 185336
Processing group:4450 hands:177960 to 178000 of 185336
Processing group:4451 hands:178000 to 178040 of 185336
Processing group:4452 hands:178040 to 178080 of 185336
Processing group:4453 hands:178080 to 178120 of 185336
Processing group:4454 hands:178120 to 178160 of 185336
Processing group:4455 hands:178160 to 178200 of 185336
Processing group:4456 hands:178200 to 178240 of 185336
Processing group:4457 hands:178240 to 178280 of 185336
Processing group:4458 hands:178280 to 178320 of 185336
Processing group:4459 hands:178320 to 178360 of 185336
Processing group:4460 hands:178360 to 178400 of 185336
Processing group:4461 hands:178400 to 178440 of 185336
Processing group:4462 hands:178440 to 178480 of 185336
Processing group:4463 hands:178480 to 178520 of 185336
Processing group:4464 hands:178520 to 178560 of 185336
Processing

Processing group:4596 hands:183800 to 183840 of 185336
Processing group:4597 hands:183840 to 183880 of 185336
Processing group:4598 hands:183880 to 183920 of 185336
Processing group:4599 hands:183920 to 183960 of 185336
Processing group:4600 hands:183960 to 184000 of 185336
Processing group:4601 hands:184000 to 184040 of 185336
Processing group:4602 hands:184040 to 184080 of 185336
Processing group:4603 hands:184080 to 184120 of 185336
Processing group:4604 hands:184120 to 184160 of 185336
Processing group:4605 hands:184160 to 184200 of 185336
Processing group:4606 hands:184200 to 184240 of 185336
Processing group:4607 hands:184240 to 184280 of 185336
Processing group:4608 hands:184280 to 184320 of 185336
Processing group:4609 hands:184320 to 184360 of 185336
Processing group:4610 hands:184360 to 184400 of 185336
Processing group:4611 hands:184400 to 184440 of 185336
Processing group:4612 hands:184440 to 184480 of 185336
Processing group:4613 hands:184480 to 184520 of 185336
Processing

In [14]:
# takes 40s
acbl_hand_records_dd_nwse_experiments_filename = 'acbl_hand_records_dd_nwse_experiments.pkl'
acbl_hand_records_dd_nwse_experiments_file = acblPath.joinpath(acbl_hand_records_dd_nwse_experiments_filename)
stdhrs.to_pickle(acbl_hand_records_dd_nwse_experiments_file)

In [15]:
stdhrs

,ref,hand_record_id,event_id,Board,EventBoard,Dealer,Vul,board_record_string,Hands,Par,Par_nwse,DDmakes,DDmakes_nwse,DD_N_C,DD_N_D,DD_N_H,DD_N_S,DD_N_N,DD_E_C,DD_E_D,DD_E_H,DD_E_S,DD_E_N,DD_S_C,DD_S_D,DD_S_H,DD_S_S,DD_S_N,DD_W_C,DD_W_D,DD_W_H,DD_W_S,DD_W_N,Hands_Bin,HB_N,HB_E,HB_S,HB_W,HCP,HCP_N,HCP_N_S,HCP_N_H,HCP_N_D,HCP_N_C,HCP_E,HCP_E_S,HCP_E_H,HCP_E_D,HCP_E_C,HCP_S,HCP_S_S,HCP_S_H,HCP_S_D,HCP_S_C,HCP_W,HCP_W_S,HCP_W_H,HCP_W_D,HCP_W_C,QT,...,CT_NS_H,CT_NS_S,CT_NS_N,CT_EW_C,CT_EW_D,CT_EW_H,CT_EW_S,CT_EW_N,CT_NS_C_Pass,CT_NS_C_Partial,CT_NS_C_Game,CT_NS_C_SSlam,CT_NS_C_GSlam,CT_NS_D_Pass,CT_NS_D_Partial,CT_NS_D_Game,CT_NS_D_SSlam,CT_NS_D_GSlam,CT_NS_H_Pass,CT_NS_H_Partial,CT_NS_H_Game,CT_NS_H_SSlam,CT_NS_H_GSlam,CT_NS_S_Pass,CT_NS_S_Partial,CT_NS_S_Game,CT_NS_S_SSlam,CT_NS_S_GSlam,CT_NS_N_Pass,CT_NS_N_Partial,CT_NS_N_Game,CT_NS_N_SSlam,CT_NS_N_GSlam,CT_EW_C_Pass,CT_EW_C_Partial,CT_EW_C_Game,CT_EW_C_SSlam,CT_EW_C_GSlam,CT_EW_D_Pass,CT_EW_D_Partial,CT_EW_D_Game,CT_EW_D_SSlam,CT_EW_D_GSlam,CT_EW_H_Pass,CT_EW_H_Partial,CT_EW_H_Game,CT_EW_H_SSlam,CT_EW_H_GSlam,CT_EW_S_Pass,CT_EW_S_Partial,CT_EW_S_Game,CT_EW_S_SSlam,CT_EW_S_GSlam,CT_EW_N_Pass,CT_EW_N_Partial,CT_EW_N_Game,CT_EW_N_SSlam,CT_EW_N_GSlam,Vul_NS,Vul_EW
0,"(acbl, 2620401)",77908,100961,09,200217A_09,N,E_W,ST6HK32DJ2CAJ9874SQJ942H765DT874CQSA853HQDKQ963CK52SK7HAJT984DA5CT63,"((T6, K32, J2, AJ9874), (A853, Q, KQ963, K52), (K7, AJT984, A5, T63), (QJ942, 765, T874, Q))","(200, [(4, S, *, EW, -1)])","(-100, [(5, C, *, S, -1)])","((10, 4, 10, 4, 8), (2, 9, 3, 9, 2), (10, 4, 10, 4, 8), (2, 9, 3, 9, 2))","((9, 3, 9, 3, 8), (2, 10, 3, 10, 2), (10, 3, 9, 3, 8), (2, 10, 3, 10, 2))",10,4,10,4,8,2,9,3,9,2,10,4,10,4,8,2,9,3,9,2,"((0b100010000010000000001100010000000011001011100100, (0b100010000, 0b100000000011, 0b1000000001...",0b100010000010000000001100010000000011001011100100,0b1000001001010001000000000001100100100100100000001001,0b100000100000100111100010010000000010000000100010010,0b11010000101000000011100000001011001000010000000000,"((9, (0, 3, 1, 5)), (14, (4, 2, 5, 3)), (12, (3, 5, 4, 0)), (5, (3, 0, 0, 2)))",9,0,3,1,5,14,4,2,5,3,12,3,5,4,0,5,3,0,0,2,"(6.5, ((1.5, (0, 0.5, 0, 1)), (2.5, (1, 0, 1, 0.5)), (2.5, (0.5, 1, 1, 0)), (0, (0, 0, 0, 0))))",...,Game,Pass,Partial,Pass,Partial,Pass,Partial,Pass,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,True
1,"(acbl, 2620412)",77908,100961,20,200217A_20,W,Both,ST85HKJDJ43CKJ873SQ32HA643DK96CT94SAJ64HT98DT75CA52SK97HQ752DAQ82CQ6,"((T85, KJ, J43, KJ873), (AJ64, T98, T75, A52), (K97, Q752, AQ82, Q6), (Q32, A643, K96, T94))","(90, [(1, D, , NS, 1), (1, C, , NS, 1), (1, N, , NS, 0)])","(90, [(1, N, , NS, 0), (1, D, , NS, 1), (1, C, , NS, 1)])","((8, 8, 7, 7, 7), (4, 5, 6, 6, 6), (8, 8, 7, 7, 7), (4, 5, 6, 6, 6))","((8, 8, 7, 7, 7), (4, 5, 6, 6, 5), (8, 8, 7, 7, 7), (4, 5, 6, 6, 5))",8,8,7,7,7,4,5,6,6,6,8,8,7,7,7,4,5,6,6,6,"((0b101001000010100000000000010000001100101001100010, (0b101001000, 0b101000000000, 0b1000000110...",0b101001000010100000000000010000001100101001100010,0b1001000010100000011100000000001001010001000000001001,0b100010100000001000010100110100010000010010000010000,0b10000000011100000001011001000100100000000110000100,"((9, (0, 4, 1, 4)), (9, (5, 0, 0, 4)), (13, (3, 2, 6, 2)), (9, (2, 4, 3, 0)))",9,0,4,1,4,9,5,0,0,4,13,3,2,6,2,9,2,4,3,0,"(6.5, ((1.0, (0, 0.5, 0, 0.5)), (2, (1, 0, 0, 1)), (2.0, (0.5, 0, 1.5, 0)), (1.5, (0, 1, 0.5, 0))))",...,Partial,Partial,Partial,Pass,Pass,Pass,Pass,Pass,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,True,True
2